In [ ]:
import pandas as pd
import csv
import re
from bs4 import BeautifulSoup
import marqo
import pprint
from tqdm import tqdm

In [ ]:
df = pd.read_csv('./data/full_one_hot_encoded.csv')
url = "http://localhost:8882"
mq = marqo.Client(url=url)

In [ ]:
df = pd.read_csv('./data/full_one_hot_encoded.csv')

def clean_text(txt):
    if pd.isna(txt):
        clean_text = "no text found"
    else:   
        soup = BeautifulSoup(str(txt).strip(), 'html.parser')
        clean_text = soup.get_text()
        clean_text = re.sub('[^a-zA-Z0-9\s]', '', clean_text)
        clean_text = clean_text.lower()
    return clean_text

df['facts_clean'] = df['facts'].apply(clean_text)
df['issue_area'] = df['issue_area'].apply(clean_text)
df['legal_question'] = df['legal_question'].apply(clean_text)
df['conclusion'] = df['conclusion'].apply(clean_text)

df['sim1_facts_vector'] = None
df['sim1_facts_vector'] = None
df['sim1_issue_area_vector'] = None
df['sim1_legal_question_vector'] = None
df['sim1_conclusion_vector'] = None
df['sim1_href'] = None
df['sim1_facts_score'] = None
df['sim1_issue_area_score'] = None
df['sim1_legal_question_score'] = None
df['sim1_conclusion_score'] = None

df['sim2_facts_vector'] = None
df['sim2_facts_vector'] = None
df['sim2_issue_area_vector'] = None
df['sim2_legal_question_vector'] = None
df['sim2_conclusion_vector'] = None
df['sim2_href'] = None
df['sim2_facts_score'] = None
df['sim2_issue_area_score'] = None
df['sim2_legal_question_score'] = None
df['sim2_conclusion_score'] = None

df['sim3_facts_vector'] = None
df['sim3_issue_area_vector'] = None
df['sim3_legal_question_vector'] = None
df['sim3_conclusion_vector'] = None
df['sim3_href'] = None
df['sim3_facts_score'] = None
df['sim3_issue_area_score'] = None
df['sim3_legal_question_score'] = None
df['sim3_conclusion_score'] = None

# df['sim1_vector_sent'] = None
# df['sim1_score_sent'] = None
# df['sim1_href_sent'] = None
# df['sim2_vector_sent'] = None
# df['sim2_score_sent'] = None
# df['sim2_href_sent'] = None
# df['sim3_vector_sent'] = None
# df['sim3_score_sent'] = None
# df['sim3_href_sent'] = None

In [ ]:
mq_idx = 'whole_legal'
for index, row in tqdm(df.iterrows()):
    search_href = row['href']
    # facts = row['facts_clean']
    # results = mq.index("whole").search(facts, search_method='TENSOR')
    # results_df = pd.DataFrame(results['hits'])
    # same_doc_index = results_df[results_df['href']==search_href].index
    # results_df.drop(same_doc_index , inplace=True)
    # results_df[0:3]

    facts = row['facts_clean']
    print("===============")
    print(facts)
    print(search_href)
    print("===============")
    results = mq.index(mq_idx).search(facts, search_method='TENSOR')
    results_df = pd.DataFrame(results['hits'])
    same_doc_index = results_df[results_df['href']==search_href].index
    results_df.drop(same_doc_index , inplace=True)
    results_df = results_df[0:3]
    print(results_df[['facts_clean', 'href', '_score']])
    df.loc[df['href']==search_href, 'sim1_facts_score'] = results_df['_score'].iloc[0]
    df.loc[df['href']==search_href, 'sim2_facts_score'] = results_df['_score'].iloc[1]
    df.loc[df['href']==search_href, 'sim3_facts_score'] = results_df['_score'].iloc[2]

    issue_area = row['issue_area']
    print("===============")
    print(issue_area)
    print(search_href)
    print("===============")
    results = mq.index(mq_idx).search(issue_area, search_method='TENSOR')
    results_df = pd.DataFrame(results['hits'])
    same_doc_index = results_df[results_df['href']==search_href].index
    results_df.drop(same_doc_index , inplace=True)
    results_df = results_df[0:3]
    print(results_df[['issue_area', 'href', '_score']])
    df.loc[df['href']==search_href, 'sim1_issue_area_score'] = results_df['_score'].iloc[0]
    df.loc[df['href']==search_href, 'sim2_issue_area_score'] = results_df['_score'].iloc[1]
    df.loc[df['href']==search_href, 'sim3_issue_area_score'] = results_df['_score'].iloc[2]

    legal_question = row['legal_question']
    print("===============")
    print(legal_question)
    print(search_href)
    print("===============")
    results = mq.index(mq_idx).search(legal_question, search_method='TENSOR')
    results_df = pd.DataFrame(results['hits'])
    same_doc_index = results_df[results_df['href']==search_href].index
    results_df.drop(same_doc_index , inplace=True)
    results_df = results_df[0:3]
    print(search_href)
    print(results_df[['legal_question', 'href', '_score']])
    df.loc[df['href']==search_href, 'sim1_legal_question_score'] = results_df['_score'].iloc[0]
    df.loc[df['href']==search_href, 'sim2_legal_question_score'] = results_df['_score'].iloc[1]
    df.loc[df['href']==search_href, 'sim3_legal_question_score'] = results_df['_score'].iloc[2]

    conclusion = row['conclusion']
    print("===============")
    print(conclusion)
    print(search_href)
    print("===============")
    results = mq.index(mq_idx).search(conclusion, search_method='TENSOR')
    results_df = pd.DataFrame(results['hits'])
    same_doc_index = results_df[results_df['href']==search_href].index
    results_df.drop(same_doc_index , inplace=True)
    results_df[0:3]
    print(search_href)
    print(results_df[['conclusion', 'href', '_score']])
    df.loc[df['href']==search_href, 'sim3_conclusion_score'] = results_df['_score'].iloc[0]
    df.loc[df['href']==search_href, 'sim3_conclusion_score'] = results_df['_score'].iloc[1]
    df.loc[df['href']==search_href, 'sim3_conclusion_score'] = results_df['_score'].iloc[2]

    for index, row in results_df[0:3].iterrows():
        document_id = row['_id']
        doc = mq.index(mq_idx).get_document(
            document_id=document_id,
            expose_facets=True
        )
                
        facets = doc['_tensor_facets']
        for i in range(len(doc['_tensor_facets'])):
            field = list(facets[i].keys())[0]
            embedding = facets[i]['_embedding']
            print(field)

        df.loc[df['href']==row['href'], f'sim{index}_{field}_vector'] = str(embedding)

In [ ]:
# for index, row in tqdm(df.iterrows()):
#     search_href = row['href']
#     facts = row['facts_clean']
#     results = mq.index("sentences").search(facts, search_method='TENSOR')
#     results_df = pd.DataFrame(results['hits'])
#     same_doc_index = results_df[results_df['href']==search_href].index
#     results_df.drop(same_doc_index , inplace=True)
#     results_df[0:3]

#     df.loc[df['href']==search_href, 'sim1_score_sent']=results_df['_score'].iloc[0]
#     df.loc[df['href']==search_href, 'sim1_href_sent'] = results_df['href'].iloc[0]
#     df.loc[df['href']==search_href, 'sim2_score_sent'] = results_df['_score'].iloc[1]
#     df.loc[df['href']==search_href, 'sim2_href_sent'] = results_df['href'].iloc[1]
#     df.loc[df['href']==search_href, 'sim3_score_sent'] = results_df['_score'].iloc[2]
#     df.loc[df['href']==search_href, 'sim3_href_sent'] = results_df['href'].iloc[2]

#     for index, row in results_df[0:3].iterrows():
#         document_id = row['_id']
#         facets = mq.index("full_orig_model_filevine_docs_sentences").get_document(
#             document_id=document_id,
#             expose_facets=True
#         )

#         df.loc[df['href']==row['href'], f'sim{index}_vector_sent'] = str(facets['_tensor_facets'][0]['_embedding'])

In [ ]:
df.to_csv('./data/final_legal.csv')

In [ ]:
# import marqo

# url = "http://localhost:8882"
# mq = marqo.Client(url=url)

# doc = mq.index("whole").get_document(
#     document_id="00295e5d-1860-45dd-96a9-e410e13fe233",
#     expose_facets=True
# )
# doc['_tensor_facets']

In [ ]:
# doc['_tensor_facets']

In [ ]:
# facets = doc['_tensor_facets']
# for index in range(len(doc['_tensor_facets'])):
#     print(index)
#     print(list(facets[index].keys())[0])
#     print(facets[index]['_embedding'])